### THE GENERATED CODE BY LLM

In [2]:
from z3 import *

def calculate_high_consumption_goods_tax(item_type, base_amount):
    """
    計算高額消費貨物的應納稅額。
    
    參數：
      - item_type: 貨物種類（例如 "小客車"、"遊艇"、"飛機/直昇機/超輕型載具"、"龜殼等"、"家具"）。
                   註：這些項目皆依【特種貨物及勞務稅條例第 7 條】，以銷售價格或完稅價格乘以10%計算應納稅額。
      - base_amount: 貨物的銷售價格或完稅價格（數值）。
    
    回傳：
      應納稅額的整數部分（例如 300000 表示 300,000 元）。
    """
    s = Solver()
    # 將輸入的基礎金額建立為 Z3 常數
    A = RealVal(base_amount)
    s.add(A == base_amount)
    
    # 應納稅額 = 基礎金額 × 10%
    tax_expr = A * 0.10
    # 取整數部分
    tax_int_expr = ToInt(tax_expr)
    
    if s.check() == sat:
        m = s.model()
        tax_int = m.evaluate(tax_int_expr, model_completion=True).as_long()
        return tax_int
    else:
        return None

def calculate_high_consumption_services_tax(sales_price):
    """
    計算高額消費勞務（入會權利）的應納稅額。
    
    參數：
      - sales_price: 該次銷售（或完稅）價格（數值）。
                   註：根據【特種貨物及勞務稅條例第 7 條】，稅率為10%，
                   此處假設銷售價格已符合課稅範圍（不包括可退還性保證金）。
    
    回傳：
      應納稅額的整數部分（例如 50000 表示 50,000 元）。
    """
    s = Solver()
    A = RealVal(sales_price)
    s.add(A == sales_price)
    
    tax_expr = A * 0.10
    tax_int_expr = ToInt(tax_expr)
    
    if s.check() == sat:
        m = s.model()
        tax_int = m.evaluate(tax_int_expr, model_completion=True).as_long()
        return tax_int
    else:
        return None

# 示範如何呼叫
if __name__ == '__main__':
    # 例如：計算「小客車」的高額消費貨物稅，
    # 若其銷售價格或完稅價格為 3,000,000 元，則應納稅額 = 3,000,000 × 10% = 300,000 元
    goods_tax = calculate_high_consumption_goods_tax("小客車", 3000000)
    print("高額消費貨物稅（小客車）：", goods_tax)
    
    # 例如：計算高額消費勞務（入會權利）的稅額，
    # 若銷售價格為 500,000 元，則應納稅額 = 500,000 × 10% = 50,000 元
    services_tax = calculate_high_consumption_services_tax(500000)
    print("高額消費勞務稅（入會權利）：", services_tax)


高額消費貨物稅（小客車）： 300000
高額消費勞務稅（入會權利）： 50000


In [10]:
import random
import time
import re
from decimal import Decimal, ROUND_HALF_UP

from z3 import Real, Solver, sat, RealVal

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import (
    ElementClickInterceptedException,
    StaleElementReferenceException,
    TimeoutException,
)

URL = "https://www.etax.nat.gov.tw/etwmain/etw158w/43"

# =========================
# 本地端稅額模型（10%）
# =========================
ROUNDING_MODE = "TRUNC"  # "TRUNC"(截尾) or "HALF_UP"(四捨五入)

ID_MAP = {
    "小客車": "highConsumption1",
    "遊艇": "highConsumption2",
    "飛機/直昇機/超輕型載具": "highConsumption3",
    "龜殼等": "highConsumption4",
    "家具": "highConsumption5",
    "入會權利": "highConsumption6",
}

GOODS_TYPES = ["小客車", "遊艇", "飛機/直昇機/超輕型載具", "龜殼等", "家具"]

GOODS_THRESHOLDS = {
    "小客車": 3_000_000,
    "遊艇": None,  # 文字條件是全長>=30.48m，但此試算頁只填金額（input2）
    "飛機/直昇機/超輕型載具": 3_000_000,
    "龜殼等": 500_000,
    "家具": 500_000,
}
SERVICES_THRESHOLD = 500_000

GOODS_RANGES = {
    "小客車": (3_000_000, 10_000_000),
    "遊艇": (500_000, 2_000_000),
    "飛機/直昇機/超輕型載具": (3_000_000, 10_000_000),
    "龜殼等": (500_000, 2_000_000),
    "家具": (500_000, 2_000_000),
}
SERVICES_RANGE = (500_000, 2_000_000)


def _rat_to_decimal(z3_rat) -> Decimal:
    num = z3_rat.numerator_as_long()
    den = z3_rat.denominator_as_long()
    return Decimal(num) / Decimal(den)

def _to_int_tax(z3_rat) -> int:
    d = _rat_to_decimal(z3_rat)
    if ROUNDING_MODE == "TRUNC":
        return int(d.to_integral_value(rounding="ROUND_DOWN"))  # 正數截尾
    return int(d.to_integral_value(rounding=ROUND_HALF_UP))

def compute_goods_tax(item_type: str, base_amount: int) -> int:
    th = GOODS_THRESHOLDS.get(item_type, None)
    if th is not None and base_amount < th:
        return 0
    s = Solver()
    tax = Real("tax")
    A = RealVal(str(int(base_amount)))
    rate = RealVal("1/10")
    s.add(tax == A * rate)
    if s.check() != sat:
        return None
    return _to_int_tax(s.model().evaluate(tax, model_completion=True))

def compute_services_tax(sales_price: int) -> int:
    if sales_price < SERVICES_THRESHOLD:
        return 0
    s = Solver()
    tax = Real("tax")
    A = RealVal(str(int(sales_price)))
    rate = RealVal("1/10")
    s.add(tax == A * rate)
    if s.check() != sat:
        return None
    return _to_int_tax(s.model().evaluate(tax, model_completion=True))


# =========================
# Selenium helpers
# =========================
def parse_int_from_text(s: str) -> int | None:
    if not s:
        return None
    s = s.strip().replace(",", "")
    m = re.search(r"-?\d+(?:\.\d+)?", s)
    if not m:
        return None
    try:
        return int(Decimal(m.group(0)).to_integral_value(rounding=ROUND_HALF_UP))
    except Exception:
        return None

def safe_click(driver, el):
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    time.sleep(0.05)
    for _ in range(3):
        try:
            el.click()
            return
        except (ElementClickInterceptedException, StaleElementReferenceException):
            try:
                driver.execute_script("arguments[0].click();", el)
                return
            except Exception:
                time.sleep(0.1)
    raise RuntimeError("click 失敗（可能被遮罩層/動畫擋住）")

def safe_clear(el):
    try:
        el.click()
        el.send_keys(Keys.CONTROL, "a")
        el.send_keys(Keys.BACKSPACE)
    except Exception:
        pass

def set_value_and_fire(driver, el, value_str: str):
    # 這頁是 Angular formcontrolname，務必觸發 input/change
    driver.execute_script(
        """
        const el = arguments[0];
        const v = arguments[1];
        el.value = v;
        el.dispatchEvent(new Event('input', {bubbles:true}));
        el.dispatchEvent(new Event('change', {bubbles:true}));
        el.dispatchEvent(new Event('blur', {bubbles:true}));
        """,
        el,
        value_str,
    )

def wait_page_ready(driver, wait: WebDriverWait):
    # 等固定 input 都出現（至少 highConsumption1）
    wait.until(EC.presence_of_element_located((By.ID, "highConsumption1")))

def get_container_btn_result_for_input(inp):
    """
    關鍵：只在「該 input 所在的最近容器」內找 button + result
    => 不會誤抓到別列、也不會碰到頁首搜尋框
    """
    container = inp.find_element(
        By.XPATH,
        "./ancestor::*[.//button[normalize-space(.)='計算' or @title='計算'] and .//span[contains(@class,'text-danger')]][1]"
    )
    btn = container.find_element(By.XPATH, ".//button[normalize-space(.)='計算' or @title='計算'][1]")
    res = container.find_element(By.XPATH, ".//span[contains(@class,'text-danger')][1]")
    return container, btn, res

def wait_result_equals(res_span, expected: int, timeout=10) -> int | None:
    """
    最穩：等結果 == expected
    （因為我們輸入都在門檻以上，expected 通常 > 0，不會卡在預設 0）
    """
    end = time.time() + timeout
    last_text = ""
    while time.time() < end:
        try:
            txt = (res_span.text or "").strip()
            last_text = txt
            v = parse_int_from_text(txt)
            if v is not None and v == expected:
                return v
        except StaleElementReferenceException:
            return None
        time.sleep(0.1)
    # timeout：回傳最後解析到的值
    return parse_int_from_text(last_text)


# =========================
# 主流程：本地 vs 網頁
# =========================
def run_special_goods_services_tax_test(total_cases=100, seed=None, headless=False, debug=False):
    if seed is not None:
        random.seed(seed)

    chrome_options = webdriver.ChromeOptions()
    if headless:
        chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--window-size=1400,900")

    driver = webdriver.Chrome(options=chrome_options)
    driver.delete_all_cookies()
    driver.get(URL)

    wait = WebDriverWait(driver, 20)
    wait_page_ready(driver, wait)
    time.sleep(0.5)

    total_tests = 0
    mismatch_tests = 0

    for i in range(total_cases):
        print(f"===== 迴圈第 {i+1} 次 =====")
        total_tests += 1

        kind = random.choice(["貨物", "勞務"])

        if kind == "貨物":
            item_type = random.choice(GOODS_TYPES)
            input_id = ID_MAP[item_type]

            inp = wait.until(EC.presence_of_element_located((By.ID, input_id)))
            _, btn, res = get_container_btn_result_for_input(inp)

            rmin, rmax = GOODS_RANGES[item_type]
            th = GOODS_THRESHOLDS[item_type] or 1
            base_amount = random.randint(max(rmin, th), rmax)

            local_tax = compute_goods_tax(item_type, base_amount)

            # 填值：先清，再 JS set + fire（比逐字輸入穩）
            safe_clear(inp)
            set_value_and_fire(driver, inp, str(base_amount))
            time.sleep(0.15)

            if debug:
                print("[DEBUG]", item_type, "input_id=", input_id, "value_now=", inp.get_attribute("value"))

            safe_click(driver, btn)
            online_tax = wait_result_equals(res, local_tax, timeout=10)

            if online_tax is None or local_tax != online_tax:
                mismatch_tests += 1
                print(">>> 不一致參數發現!")
                print(f"Kind: 貨物, item_type: {item_type}")
                print(f"base_amount: {base_amount}")
                print(f"本地稅額: {local_tax}, 線上稅額: {online_tax}")
            else:
                print("比對一致。")

        else:
            input_id = ID_MAP["入會權利"]
            inp = wait.until(EC.presence_of_element_located((By.ID, input_id)))
            _, btn, res = get_container_btn_result_for_input(inp)

            sales_price = random.randint(max(SERVICES_RANGE[0], SERVICES_THRESHOLD), SERVICES_RANGE[1])
            local_tax = compute_services_tax(sales_price)

            safe_clear(inp)
            set_value_and_fire(driver, inp, str(sales_price))
            time.sleep(0.15)

            if debug:
                print("[DEBUG] 入會權利 input_id=", input_id, "value_now=", inp.get_attribute("value"))

            safe_click(driver, btn)
            online_tax = wait_result_equals(res, local_tax, timeout=10)

            if online_tax is None or local_tax != online_tax:
                mismatch_tests += 1
                print(">>> 不一致參數發現!")
                print("Kind: 勞務（入會權利）")
                print(f"sales_price: {sales_price}")
                print(f"本地稅額: {local_tax}, 線上稅額: {online_tax}")
            else:
                print("比對一致。")

        # 清空方式：直接 refresh 最乾淨（也避免殘值影響）
        driver.refresh()
        time.sleep(1.2)
        wait_page_ready(driver, wait)

    print(f"總測試次數: {total_tests}, 不一致次數: {mismatch_tests}")
    driver.quit()


if __name__ == "__main__":
    run_special_goods_services_tax_test(total_cases=100, seed=42, headless=False, debug=False)


===== 迴圈第 1 次 =====
比對一致。
===== 迴圈第 2 次 =====
比對一致。
===== 迴圈第 3 次 =====
比對一致。
===== 迴圈第 4 次 =====
比對一致。
===== 迴圈第 5 次 =====
比對一致。
===== 迴圈第 6 次 =====
比對一致。
===== 迴圈第 7 次 =====
比對一致。
===== 迴圈第 8 次 =====
比對一致。
===== 迴圈第 9 次 =====
比對一致。
===== 迴圈第 10 次 =====
比對一致。
===== 迴圈第 11 次 =====
比對一致。
===== 迴圈第 12 次 =====
比對一致。
===== 迴圈第 13 次 =====
比對一致。
===== 迴圈第 14 次 =====
比對一致。
===== 迴圈第 15 次 =====
比對一致。
===== 迴圈第 16 次 =====
比對一致。
===== 迴圈第 17 次 =====
比對一致。
===== 迴圈第 18 次 =====
比對一致。
===== 迴圈第 19 次 =====
比對一致。
===== 迴圈第 20 次 =====
比對一致。
===== 迴圈第 21 次 =====
比對一致。
===== 迴圈第 22 次 =====
比對一致。
===== 迴圈第 23 次 =====
比對一致。
===== 迴圈第 24 次 =====
比對一致。
===== 迴圈第 25 次 =====
比對一致。
===== 迴圈第 26 次 =====
比對一致。
===== 迴圈第 27 次 =====
比對一致。
===== 迴圈第 28 次 =====
比對一致。
===== 迴圈第 29 次 =====
比對一致。
===== 迴圈第 30 次 =====
比對一致。
===== 迴圈第 31 次 =====
比對一致。
===== 迴圈第 32 次 =====
比對一致。
===== 迴圈第 33 次 =====
比對一致。
===== 迴圈第 34 次 =====
比對一致。
===== 迴圈第 35 次 =====
比對一致。
===== 迴圈第 36 次 =====
比對一致。
===== 迴圈第 37 次 =====
比對一致。
===== 迴圈第 

In [11]:
import random
import time
import re
from decimal import Decimal, ROUND_HALF_UP

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException

URL = "https://www.etax.nat.gov.tw/etwmain/etw158w/43"

# ===== 你提供的固定 input id mapping =====
ID_MAP = {
    "小客車": "highConsumption1",
    "遊艇": "highConsumption2",
    "飛機/直昇機/超輕型載具": "highConsumption3",
    "龜殼等": "highConsumption4",
    "家具": "highConsumption5",
    "入會權利": "highConsumption6",
}
GOODS_TYPES = ["小客車", "遊艇", "飛機/直昇機/超輕型載具", "龜殼等", "家具"]

# 測資範圍：用門檻當下限（避免網站回 0）
GOODS_RANGES = {
    "小客車": (3_000_000, 10_000_000),
    "遊艇": (500_000, 2_000_000),
    "飛機/直昇機/超輕型載具": (3_000_000, 10_000_000),
    "龜殼等": (500_000, 2_000_000),
    "家具": (500_000, 2_000_000),
}
SERVICES_RANGE = (500_000, 2_000_000)


# =========================
# Selenium helpers
# =========================
def parse_int_from_text(s: str) -> int | None:
    if not s:
        return None
    s = s.strip().replace(",", "")
    m = re.search(r"-?\d+(?:\.\d+)?", s)
    if not m:
        return None
    try:
        return int(Decimal(m.group(0)).to_integral_value(rounding=ROUND_HALF_UP))
    except Exception:
        return None

def safe_click(driver, el):
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    time.sleep(0.05)
    for _ in range(3):
        try:
            el.click()
            return
        except (ElementClickInterceptedException, StaleElementReferenceException):
            try:
                driver.execute_script("arguments[0].click();", el)
                return
            except Exception:
                time.sleep(0.1)
    raise RuntimeError("click 失敗（可能被遮罩層/動畫擋住）")

def safe_clear(el):
    try:
        el.click()
        el.send_keys(Keys.CONTROL, "a")
        el.send_keys(Keys.BACKSPACE)
    except Exception:
        pass

def set_value_and_fire(driver, el, value_str: str):
    # Angular formcontrolname：用 JS set + input/change/blur 最穩
    driver.execute_script(
        """
        const el = arguments[0];
        const v = arguments[1];
        el.value = v;
        el.dispatchEvent(new Event('input', {bubbles:true}));
        el.dispatchEvent(new Event('change', {bubbles:true}));
        el.dispatchEvent(new Event('blur', {bubbles:true}));
        """,
        el,
        value_str,
    )

def wait_page_ready(driver, wait: WebDriverWait):
    wait.until(EC.presence_of_element_located((By.ID, "highConsumption1")))

def get_btn_and_result_span_for_input(inp):
    """
    在「該 input 最近的容器」內找：
      - 計算 button
      - 結果 span.text-danger
    只在容器內找 => 不會誤抓到別列，也不會打到頁首搜尋框
    """
    container = inp.find_element(
        By.XPATH,
        "./ancestor::*[.//button[normalize-space(.)='計算' or @title='計算']][1]"
    )

    # 計算按鈕：容器內第一顆「計算」
    btn = container.find_element(By.XPATH, ".//button[normalize-space(.)='計算' or @title='計算'][1]")

    # 結果：容器內第一個可見 text-danger（多數情況就是那個 $紅字）
    spans = container.find_elements(By.CSS_SELECTOR, "span.text-danger")
    spans = [s for s in spans if s.is_displayed()]
    if not spans:
        # fallback：找任何 .text-danger
        spans = container.find_elements(By.CSS_SELECTOR, ".text-danger")
        spans = [s for s in spans if s.is_displayed()]
    if not spans:
        raise RuntimeError("找不到結果 span.text-danger（DOM 可能變了）")

    return btn, spans[0]

def wait_result_equals(res_span, expected: int, timeout=10) -> int | None:
    """
    等結果變成 expected（避免拿到預設 0）
    """
    end = time.time() + timeout
    last_text = ""
    while time.time() < end:
        try:
            txt = (res_span.text or "").strip()
            last_text = txt
            v = parse_int_from_text(txt)
            if v is not None and v == expected:
                return v
        except StaleElementReferenceException:
            return None
        time.sleep(0.1)
    return parse_int_from_text(last_text)


# =========================
# 主流程：本地（直接 call 你 notebook 內定義的函式） vs 網頁
# =========================
def run_special_goods_services_tax_test(total_cases=100, seed=None, headless=False, debug=False):
    # 1) 確保你已在 notebook 先定義好本地函式
    if "calculate_high_consumption_goods_tax" not in globals():
        raise RuntimeError("你還沒在 notebook 定義 calculate_high_consumption_goods_tax")
    if "calculate_high_consumption_services_tax" not in globals():
        raise RuntimeError("你還沒在 notebook 定義 calculate_high_consumption_services_tax")

    if seed is not None:
        random.seed(seed)

    chrome_options = webdriver.ChromeOptions()
    if headless:
        chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--window-size=1400,900")

    driver = webdriver.Chrome(options=chrome_options)
    driver.delete_all_cookies()
    driver.get(URL)

    wait = WebDriverWait(driver, 20)
    wait_page_ready(driver, wait)
    time.sleep(0.4)

    total_tests = 0
    mismatch_tests = 0

    for i in range(total_cases):
        print(f"===== 迴圈第 {i+1} 次 =====")
        total_tests += 1

        kind = random.choice(["貨物", "勞務"])

        if kind == "貨物":
            item_type = random.choice(GOODS_TYPES)
            input_id = ID_MAP[item_type]

            inp = wait.until(EC.presence_of_element_located((By.ID, input_id)))
            btn, res = get_btn_and_result_span_for_input(inp)

            lo, hi = GOODS_RANGES[item_type]
            base_amount = random.randint(lo, hi)

            # ✅ 本地：直接 call 你 notebook 已生成的函式
            local_tax = calculate_high_consumption_goods_tax(item_type, base_amount)
            if local_tax is None:
                raise RuntimeError("本地 goods tax 回傳 None（請檢查本地函式）")
            local_tax = int(local_tax)

            safe_clear(inp)
            set_value_and_fire(driver, inp, str(base_amount))
            time.sleep(0.15)

            if debug:
                print("[DEBUG]", item_type, "id=", input_id, "value_now=", inp.get_attribute("value"), "local_tax=", local_tax)

            safe_click(driver, btn)
            online_tax = wait_result_equals(res, local_tax, timeout=10)

            if online_tax is None or local_tax != online_tax:
                mismatch_tests += 1
                print(">>> 不一致參數發現!")
                print(f"Kind: 貨物, item_type: {item_type}")
                print(f"base_amount: {base_amount}")
                print(f"本地稅額: {local_tax}, 線上稅額: {online_tax}")
            else:
                print("比對一致。")

        else:
            input_id = ID_MAP["入會權利"]
            inp = wait.until(EC.presence_of_element_located((By.ID, input_id)))
            btn, res = get_btn_and_result_span_for_input(inp)

            sales_price = random.randint(SERVICES_RANGE[0], SERVICES_RANGE[1])

            # ✅ 本地：直接 call 你 notebook 已生成的函式
            local_tax = calculate_high_consumption_services_tax(sales_price)
            if local_tax is None:
                raise RuntimeError("本地 services tax 回傳 None（請檢查本地函式）")
            local_tax = int(local_tax)

            safe_clear(inp)
            set_value_and_fire(driver, inp, str(sales_price))
            time.sleep(0.15)

            if debug:
                print("[DEBUG] 入會權利 id=", input_id, "value_now=", inp.get_attribute("value"), "local_tax=", local_tax)

            safe_click(driver, btn)
            online_tax = wait_result_equals(res, local_tax, timeout=10)

            if online_tax is None or local_tax != online_tax:
                mismatch_tests += 1
                print(">>> 不一致參數發現!")
                print("Kind: 勞務（入會權利）")
                print(f"sales_price: {sales_price}")
                print(f"本地稅額: {local_tax}, 線上稅額: {online_tax}")
            else:
                print("比對一致。")

        driver.refresh()
        time.sleep(1.0)
        wait_page_ready(driver, wait)

    print(f"總測試次數: {total_tests}, 不一致次數: {mismatch_tests}")
    driver.quit()


# === 直接跑 ===
run_special_goods_services_tax_test(total_cases=100, seed=42, headless=False, debug=False)


===== 迴圈第 1 次 =====
比對一致。
===== 迴圈第 2 次 =====
比對一致。
===== 迴圈第 3 次 =====
比對一致。
===== 迴圈第 4 次 =====
比對一致。
===== 迴圈第 5 次 =====
比對一致。
===== 迴圈第 6 次 =====
比對一致。
===== 迴圈第 7 次 =====
比對一致。
===== 迴圈第 8 次 =====
比對一致。
===== 迴圈第 9 次 =====
比對一致。
===== 迴圈第 10 次 =====
比對一致。
===== 迴圈第 11 次 =====
比對一致。
===== 迴圈第 12 次 =====
比對一致。
===== 迴圈第 13 次 =====
比對一致。
===== 迴圈第 14 次 =====
比對一致。
===== 迴圈第 15 次 =====
比對一致。
===== 迴圈第 16 次 =====
比對一致。
===== 迴圈第 17 次 =====
比對一致。
===== 迴圈第 18 次 =====
比對一致。
===== 迴圈第 19 次 =====
比對一致。
===== 迴圈第 20 次 =====
比對一致。
===== 迴圈第 21 次 =====
比對一致。
===== 迴圈第 22 次 =====
比對一致。
===== 迴圈第 23 次 =====
比對一致。
===== 迴圈第 24 次 =====
比對一致。
===== 迴圈第 25 次 =====
比對一致。
===== 迴圈第 26 次 =====
比對一致。
===== 迴圈第 27 次 =====
比對一致。
===== 迴圈第 28 次 =====
比對一致。
===== 迴圈第 29 次 =====
比對一致。
===== 迴圈第 30 次 =====
比對一致。
===== 迴圈第 31 次 =====
比對一致。
===== 迴圈第 32 次 =====
比對一致。
===== 迴圈第 33 次 =====
比對一致。
===== 迴圈第 34 次 =====
比對一致。
===== 迴圈第 35 次 =====
比對一致。
===== 迴圈第 36 次 =====
比對一致。
===== 迴圈第 37 次 =====
比對一致。
===== 迴圈第 

### According to the selenium test above, we have 0 mismatches against the MoF protal